In [8]:
import pandas as pd

file_path = "./p5_reference.txt"

df = pd.read_csv(file_path, sep="\n")

functions = []
headers = []
for item in df.values.tolist() :
    ele = item[0].strip()
    if(ele[0].isupper()) :
        headers.append(ele)
    else :
        ele = item[0].strip()
        functions.append(ele)

functions


['describe()',
 'describeElement()',
 'textOutput()',
 'gridOutput()',
 'print()',
 'frameCount',
 'deltaTime',
 'focused',
 'cursor()',
 'frameRate()',
 'noCursor()',
 'displayWidth',
 'displayHeight',
 'windowWidth',
 'windowHeight',
 'windowResized()',
 'width',
 'height',
 'fullscreen()',
 'pixelDensity()',
 'displayDensity()',
 'getURL()',
 'getURLPath()',
 'getURLParams()',
 'alpha()',
 'blue()',
 'brightness()',
 'color()',
 'green()',
 'hue()',
 'lerpColor()',
 'lightness()',
 'red()',
 'saturation()',
 'p5.Color',
 'background()',
 'clear()',
 'colorMode()',
 'fill()',
 'noFill()',
 'noStroke()',
 'stroke()',
 'erase()',
 'noErase()',
 '2D Primitives',
 'arc()',
 'ellipse()',
 'circle()',
 'line()',
 'point()',
 'quad()',
 'rect()',
 'square()',
 'triangle()',
 'ellipseMode()',
 'noSmooth()',
 'rectMode()',
 'smooth()',
 'strokeCap()',
 'strokeJoin()',
 'strokeWeight()',
 'bezier()',
 'bezierDetail()',
 'bezierPoint()',
 'bezierTangent()',
 'curve()',
 'curveDetail()',
 'curve